In [ ]:
'''imports'''
'''This file is used for the prediction of stock prices based upon the dataset located here:
    https://www.kaggle.com/datasets/borismarjanovic/price-volume-data-for-all-us-stocks-etfs 
    data import, model training, and model testing will all be included in this file
 '''
import sklearn as sk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
'''importing the data'''
df = pd.read_csv('../Datasets/StockPricesData/Stocks/a.us.txt')